In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import state_union
from nltk.tokenize import  PunktSentenceTokenizer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
data= pd.read_csv("text_training.csv", delimiter="|").dropna()

In [3]:
print (data.shape)
source = unicode(data["Review Text"][0], 'utf-8')
data= data.reindex(np.random.permutation(data.index))

(47999, 3)


In [4]:
len((data[data["Category"]==3]))

11370

In [5]:
x_train=data["Review Text"]
"""from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x)
X_train_counts.shape"""

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, max_features= None) #, ngram_range=(1,2)
x_data=vectorizer.fit_transform(x_train)


In [6]:
Y_train=data["Category"]
X_tr=x_data
Y_tr=Y_train
X_test=vectorizer.transform(data["Review Text"][42001:])
Y_test=Y_train[42001:]



In [7]:
X_test.shape


(5998, 48597)

In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV


#clf=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)


In [9]:
clf=SGDClassifier()
parameters={"alpha":np.linspace(0.00000001,50, num=100),
           "penalty":("l1", 'l2')
           }
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf=gs_clf.fit(X_tr, Y_tr)


In [10]:
gs_clf.best_score_



0.88493510281464194

In [12]:
clf=svm.LinearSVC() 
parameters={"C":np.linspace(0.0000001,100)}
gs_lsvc_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_lsvc_clf=gs_lsvc_clf.fit(X_tr, Y_tr)

In [ ]:
#clf=svm.SVC(kernel="poly") #0.87121780445111274
#clf=linear_model.LogisticRegression(penalty="l1") #0.86271567891972989
clf=svm.SVC(kernel="poly") 
parameters={"C":np.linspace(0.0000001,50)}
gs_lsvc_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_lsvc_clf=gs_lsvc_clf.fit(X_tr, Y_tr)

In [89]:
predicted=gs_lsvc_clf.predict(X_test)
accuracy= np.mean(predicted==Y_test)

In [13]:
gs_lsvc_clf.best_score_

0.90351882330881894

In [24]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB(fit_prior=True)
parameters={"alpha":np.linspace(0.000000001,100, num=500)}
gs_nb_clf=GridSearchCV(clf, parameters, n_jobs=-1)
gs_nb_clf=gs_nb_clf.fit(X_tr, Y_tr)

In [25]:
gs_nb_clf.best_score_

0.86326798474968225

In [27]:
data_kaggle=pd.read_csv("test.csv",delimiter="|")

In [28]:
data_kaggle.head()
kaggle_text=data_kaggle["Review Text"]
kaggle_processed= vectorizer.transform(kaggle_text)


In [29]:
kaggle_predict=gs_clf.predict(kaggle_processed)

In [30]:
submission=pd.DataFrame({"ID": data_kaggle["ID"]})
submission["Category"]=kaggle_predict
submission["ID"].shape

(12000,)

In [31]:
submission.to_csv("submission.csv")

In [ ]:
def lemma(sents): #Lemmatizing the  words 
    tagged_POS_sents = [nltk.pos_tag(sent) for sent in sents]
    lemma= WordNetLemmatizer()
    word_lemma=[]
    for sent in tagged_POS_sents:
        intermediary=[]
        for word in sent: # Lemmatising according to the POS to maintain integrity of meaning
            if word[1][:2] == "NN" or word[1][:2] == "JJ" or word[1][:2] == "VB" or word[1][:2] == "RB":  
                lemma_tag = {'NN':wordnet.NOUN,'JJ':wordnet.ADJ,'VB':wordnet.VERB,'RB':wordnet.ADV}
                intermediary.append(lemma.lemmatize(word[0], pos=lemma_tag[word[1][:2]]))
            else:
               intermediary.append(lemma.lemmatize(word[0]))
        word_lemma.append(intermediary)
    return word_lemma
                                  


1+1

In [ ]:
a= lemma(x_label)nf = 4
pca = PCA(n_components=nf)
# X is the matrix transposed (n samples on the rows, m features on the columns)
pca.fit(X_tr.toarray())

X_new = pca.transform(X_tr.toarray())

In [1]:
1+1

2